In [1]:
import bambi as bmb
import numpy as np
import pandas as pd

In [2]:
data = bmb.load_data("sleepstudy")

In [3]:
data.head()

,Reaction,Days,Subject
0,249.5600,0,308
1,258.7047,1,308
2,250.8006,2,308
3,321.4398,3,308
4,356.8519,4,308


In [4]:
model = bmb.Model("Reaction ~ 1 + Days + (1 + Days | Subject)", data)
model

       Formula: Reaction ~ 1 + Days + (1 + Days | Subject)
        Family: gaussian
          Link: mu = identity
  Observations: 180
        Priors: 
    target = mu
        Common-level effects
            Intercept ~ Normal(mu: 298.5079, sigma: 261.0092)
            Days ~ Normal(mu: 0.0, sigma: 48.8915)
        
        Group-level effects
            1|Subject ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 261.0092))
            Days|Subject ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 48.8915))
        
        Auxiliary parameters
            sigma ~ HalfStudentT(nu: 4.0, sigma: 56.1721)

In [5]:
idata = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Reaction_sigma, Intercept, Days, 1|Subject_sigma, 1|Subject_offset, Days|Subject_sigma, Days|Subject_offset]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 17 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


**Problema:** No estamos guardando la correlacion entre los parametros. No podriamos generar muestras con el metodo "gaussian" porque estariamos omitiendo la correlacion.

Una alternativa seria computar esa correlacion...

In [6]:
import formulae
from formulae import design_matrices

formulae.config["EVAL_UNSEEN_CATEGORIES"] = "silent"

dm = design_matrices("Reaction ~ 1 + Days + (1 + Days|Subject)", data)

df_new = data.head(10).reset_index(drop=True)
df_new["Subject"] = "xxx"
df_new = pd.concat([df_new, data.head(10)])
df_new = df_new.reset_index(drop=True)

dm_new = dm.group.evaluate_new_data(df_new)
dm_new

GroupEffectsMatrix with shape (20, 38)
Terms:  
  1|Subject  
    kind: intercept
    groups: ['308', '309', '310', '330', '331', '332', '333', '334', '335', '337', '349', '350',
      '351', '352', '369', '370', '371', '372', '__NEW_FACTOR_GROUP__']
    columns: 0:19
  Days|Subject  
    kind: numeric
    groups: ['308', '309', '310', '330', '331', '332', '333', '334', '335', '337', '349', '350',
      '351', '352', '369', '370', '371', '372', '__NEW_FACTOR_GROUP__']
    columns: 19:38

To access the actual design matrix do 'np.array(this_obj)'

In [7]:
import xarray as xr

In [8]:
ndraws = len(idata.posterior.coords["draw"])
factor_dim = "Subject__factor_dim"

factor_levels = idata.posterior.coords[factor_dim].to_numpy()
factor_sampled_idxs = np.random.choice(np.arange(len(factor_levels)), size=ndraws)

original_draws_1 = idata.posterior["1|Subject"].to_numpy()
new_group_draws_1 = original_draws_1[:, np.arange(ndraws), factor_sampled_idxs]

original_draws_2 = idata.posterior["Days|Subject"].to_numpy()
new_group_draws_2 = original_draws_2[:, np.arange(ndraws), factor_sampled_idxs]

coords = {
    "chain": np.arange(2), 
    "draw": np.arange(ndraws), 
    "Subject__factor_dim": ["__NEW_FACTOR_GROUP__"]
}
new_group_draws_1 = xr.DataArray(new_group_draws_1[..., np.newaxis], coords=coords)
new_group_draws_2 = xr.DataArray(new_group_draws_2[..., np.newaxis], coords=coords)

In [9]:
z1 = xr.concat([idata.posterior["1|Subject"], new_group_draws_1], dim="Subject__factor_dim")
z2 = xr.concat([idata.posterior["Days|Subject"], new_group_draws_2], dim="Subject__factor_dim")

u = xr.Dataset({"1|Subject": z1, "Days|Subject": z2})
u = u.to_stacked_array("__variables__", ("chain", "draw"))

In [10]:
response_dim = "Reaction_obs"
design_matrix_dims = (response_dim, "__variables__")

Z = np.asarray(dm_new)
Z = xr.DataArray(Z, dims=design_matrix_dims)
xr.dot(Z, u)

<xarray.DataArray (Reaction_obs: 20, chain: 2, draw: 1000)>
array([[[ 26.83926821, -36.04168089,  -9.35326398, ...,   7.74448664,
          -2.94360742,  28.19351309],
        [ 12.88274655, -35.42117736, -16.38678506, ...,  -3.51445808,
           1.8916228 ,  22.46153754]],

       [[ 24.66897939, -45.68448   , -21.20078053, ...,   4.91612669,
           0.96185138,  23.84984079],
        [ 10.46990251, -42.90295421, -29.21691552, ...,  -6.40808743,
           3.67777385,  18.12596928]],

       [[ 22.49869057, -55.32727911, -33.04829709, ...,   2.08776675,
           4.86731017,  19.5061685 ],
        [  8.05705846, -50.38473106, -42.04704598, ...,  -9.30171678,
           5.46392489,  13.79040103]],

       ...,

       [[ 79.29615641,  52.94744274,  57.53621111, ...,  85.82364142,
          68.03346272,  55.12011374],
        [ 57.80208435,  75.75276117,  72.36944344, ...,  44.35252731,
          60.05172602,  59.50508663]],

       [[ 92.05378078,  57.10679132,  69.99540819, ...,  97.00290182,
          75.95879148,  58.43544833],
        [ 64.01465784,  87.77503988,  81.65270335, ...,  52.51160674,
          64.8607413 ,  70.49974574]],

       [[104.81140516,  61.26613991,  82.45460527, ..., 108.18216222,
          83.88412023,  61.75078292],
        [ 70.22723134,  99.79731858,  90.93596325, ...,  60.67068617,
          69.66975659,  81.49440484]]])
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Dimensions without coordinates: Reaction_obs

In [11]:
print(np.array(dm_new))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 4]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 5]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 6]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 7]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 8]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 9]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 

Necesitamos otros cambios a nivel de formulae

* Si hay observaciones nuevas, se agrega un nuevo grupo, a la derecha de los grupos existentes.
    * Todos los nuevos grupos son interpretados como UN UNICO nuevo grupo
    * No va a tener sentido para los efectos fijos
    * Si va a tener sentido para los efectos aleatorios
* Será mas fácil cuando se tengan que hacer predicciones para un nuevo grupo.
    * Simplemente "apendeamos" 